In [1]:
import pandas as pd

In [2]:
# Reading in all files
# events_England x matches_England on (match_id & team id) 
df_eng_events = pd.read_csv('./data/events_England.csv')
df_eng_matches = pd.read_csv('./data/matches_England.csv')
df_teams = pd.read_csv('./data/teams.csv')
df_players = pd.read_csv('./data/players.csv')
df_activities = pd.read_csv('./data/actions.csv')

In [3]:
# Dataframe columns
print('df_activities columns:', df_activities.columns)
print('df_eng_matches columns:', df_eng_matches.columns)
print('df_eng_events columns:', df_eng_events.columns)

df_activities columns: Index(['Unnamed: 0', 'game_id', 'period_id', 'time_seconds', 'team_id',
       'player_id', 'start_x', 'start_y', 'end_x', 'end_y',
       'original_event_id', 'bodypart_id', 'type_id', 'result_id', 'action_id',
       'type_name', 'result_name', 'bodypart_name'],
      dtype='object')
df_eng_matches columns: Index(['status', 'roundId', 'gameweek', 'teamsData', 'seasonId', 'dateutc',
       'winner', 'venue', 'wyId', 'label', 'date', 'referees', 'duration',
       'competitionId', 'team1.scoreET', 'team1.coachId', 'team1.side',
       'team1.teamId', 'team1.score', 'team1.scoreP', 'team1.hasFormation',
       'team1.formation', 'team1.scoreHT', 'team1.formation.bench',
       'team1.formation.lineup', 'team1.formation.substitutions',
       'team2.scoreET', 'team2.coachId', 'team2.side', 'team2.teamId',
       'team2.score', 'team2.scoreP', 'team2.hasFormation', 'team2.formation',
       'team2.scoreHT', 'team2.formation.bench', 'team2.formation.lineup',
       '

In [4]:
# Dropping unnecessary columns
df_eng_events = df_eng_events.drop(['positions','subEventId','tags'], axis=1)

In [5]:
# Table merging
df_events_teams = pd.merge(df_eng_events, df_teams[['wyId','name']], left_on = 'teamId', right_on = 'wyId')
df_events_teams_matches = pd.merge(df_events_teams, df_eng_matches[['wyId', 'team1.teamId','team2.teamId','team1.score','team2.score']], left_on = 'matchId', right_on = 'wyId')
df_events_teams_matches = pd.merge(df_events_teams_matches, df_teams[['wyId','name']], left_on = 'team2.teamId', right_on = 'wyId')
df_ev_tm_mt_plyrs = pd.merge(df_events_teams_matches, df_players[['wyId','firstName','lastName']], left_on='playerId', right_on='wyId')

/var/folders/b9/qfzh83d95370qmxpxprblp5r0000gp/T/ipykernel_62041/1445828198.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'wyId_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_ev_tm_mt_plyrs = pd.merge(df_events_teams_matches, df_players[['wyId','firstName','lastName']], left_on='playerId', right_on='wyId')


In [6]:
# Reducing dataset to passes and shots. Change this to include additional actions.
df_passes = df_ev_tm_mt_plyrs.query('eventId == 8 | eventId == 10')

In [7]:
# Dropping more columns
# df_passes = df_passes.drop(['eventId','tags', 'positions', 'teamId', 'id', 'subEventId', 'wyId_x', 'wyId_y', 'wyId_x', 'wyId_y'], axis=1)
df_passes = df_passes.drop(['eventId', 'teamId', 'wyId_x', 'wyId_y', 'wyId_x', 'wyId_y'], axis=1)
print('df_passes columns:', df_passes.columns)

df_passes columns: Index(['subEventName', 'playerId', 'matchId', 'eventName', 'matchPeriod',
       'eventSec', 'id', 'tagsList', 'pos_orig_y', 'pos_orig_x', 'pos_dest_y',
       'pos_dest_x', 'name_x', 'team1.teamId', 'team2.teamId', 'team1.score',
       'team2.score', 'name_y', 'firstName', 'lastName'],
      dtype='object')


In [8]:
# Merging new x,y values from activities.csv

# Passes and interceptions have same original_event_id
df_activities_passes = df_activities[df_activities['type_name'] != 'interception']

# Merging
df_activities_merged = pd.merge(df_passes, df_activities_passes[['start_x','start_y', 'end_x','end_y','original_event_id', 'result_name']], left_on = 'id', right_on = 'original_event_id',copy=False)

# Dropping old x,y values
df_activities_merged = df_activities_merged.drop(['pos_orig_y', 'pos_orig_x', 'pos_dest_y','pos_dest_x', 'original_event_id', 'id'], axis=1)
df_activities_merged.head()

,subEventName,playerId,matchId,eventName,matchPeriod,eventSec,tagsList,name_x,team1.teamId,team2.teamId,team1.score,team2.score,name_y,firstName,lastName,start_x,start_y,end_x,end_y,result_name
0,Simple pass,25413,2499719,Pass,1H,2.758649,[1801],Arsenal,1609,1631,4,3,Leicester City,Alexandre,Lacazette,53.55,33.32,72.45,53.04,success
1,Shot,25413,2499719,Shot,1H,94.595788,"[101, 402, 201, 1205, 1801]",Arsenal,1609,1631,4,3,Leicester City,Alexandre,Lacazette,12.60,27.88,0.00,30.60,success
2,Head pass,25413,2499719,Pass,1H,397.881307,[1802],Arsenal,1609,1631,4,3,Leicester City,Alexandre,Lacazette,30.45,47.60,30.45,42.84,fail
3,Simple pass,25413,2499719,Pass,1H,752.105957,[1802],Arsenal,1609,1631,4,3,Leicester City,Alexandre,Lacazette,73.50,25.84,75.60,0.00,fail
4,Simple pass,25413,2499719,Pass,1H,1253.590895,[1801],Arsenal,1609,1631,4,3,Leicester City,Alexandre,Lacazette,29.40,22.44,40.95,15.64,success


In [9]:
# Confirming shapes match after merge
print('shape of original: ', df_passes.shape)
print('shape of merged: ', df_activities_merged.shape)

shape of original:  (337067, 20)
shape of merged:  (337067, 20)


In [10]:
df_activities_merged.columns

Index(['subEventName', 'playerId', 'matchId', 'eventName', 'matchPeriod',
       'eventSec', 'tagsList', 'name_x', 'team1.teamId', 'team2.teamId',
       'team1.score', 'team2.score', 'name_y', 'firstName', 'lastName',
       'start_x', 'start_y', 'end_x', 'end_y', 'result_name'],
      dtype='object')

In [11]:
# Renaming columns
df_activities_merged = df_activities_merged.rename(columns={"name_x": "team1.name", "name_y": "team2.name"})
df_activities_merged.head()

,subEventName,playerId,matchId,eventName,matchPeriod,eventSec,tagsList,team1.name,team1.teamId,team2.teamId,team1.score,team2.score,team2.name,firstName,lastName,start_x,start_y,end_x,end_y,result_name
0,Simple pass,25413,2499719,Pass,1H,2.758649,[1801],Arsenal,1609,1631,4,3,Leicester City,Alexandre,Lacazette,53.55,33.32,72.45,53.04,success
1,Shot,25413,2499719,Shot,1H,94.595788,"[101, 402, 201, 1205, 1801]",Arsenal,1609,1631,4,3,Leicester City,Alexandre,Lacazette,12.60,27.88,0.00,30.60,success
2,Head pass,25413,2499719,Pass,1H,397.881307,[1802],Arsenal,1609,1631,4,3,Leicester City,Alexandre,Lacazette,30.45,47.60,30.45,42.84,fail
3,Simple pass,25413,2499719,Pass,1H,752.105957,[1802],Arsenal,1609,1631,4,3,Leicester City,Alexandre,Lacazette,73.50,25.84,75.60,0.00,fail
4,Simple pass,25413,2499719,Pass,1H,1253.590895,[1801],Arsenal,1609,1631,4,3,Leicester City,Alexandre,Lacazette,29.40,22.44,40.95,15.64,success


In [12]:
df_activities_merged.to_csv('./data/initial_clean_and_join.csv')